In [ ]:
from ontobio.ontol_factory import OntologyFactory
import pickle
import pandas as pd
from collections import Counter
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import re

In [ ]:
ont = OntologyFactory().create('hp.obo')

In [ ]:
# expand: repeat hp term across set by its frequency
def hp_exp_by_freq(model_df):
    exp_d = []
    for i in range(len(model_df)):
        mod_id = model_df.loc[i]['model_id']
        # repeat each hp term according to its freq (in nested list)
        exp_lists = [[hp for i in range(f)] for hp, f in model_df.loc[i]['hp_freq']]
        # convert nested list to single list
        exp_hp = [hp for hp_list in exp_lists for hp in hp_list]
        for hp in exp_hp:
            exp_d.append({'model_id':mod_id, 'hp':hp})
    return exp_d

# each hp term just by unique occurence per model
def hp_base_df(model_df):
    base_d = []
    for i in range(len(model_df)):
        mod_id = model_df.loc[i]['model_id']
        for hp,v in Counter(model_df.loc[i]['hp_base']).most_common():
            base_d.append(({'model_id':mod_id, 'hp':hp, 'model_freq': v}))
    return pd.DataFrame(base_d)

# add ancestors for each annotated term
def anc_freq_df(hp_freq_df):
    anc_d = []
    for i in range(len(hp_freq_df)):
        mod_id = hp_freq_df.loc[i]['model_id']
        hp_base_t = hp_freq_df.loc[i]['hp']
        hp_anc = ont.ancestors(hp_base_t)
        hp_anc.insert(0, hp_base_t)
        for hp_prop in hp_anc:

            anc_d.append({'model_id':mod_id, 'hp':hp_prop})
                #     drop duplicates for each model
    return(pd.DataFrame(anc_d).drop_duplicates())

# count occurences of hp term across set and divide by no dis models to get base freq
def ic_df(hp_freq_df):
    n_models = len(set(hp_freq_df['model_id']))
    base_freq_df = pd.DataFrame(hp_freq_df['hp'].value_counts()).reset_index()
    base_freq_df = base_freq_df.rename(columns={'index':'hp', 'hp':'term_freq'})
    base_freq_df['anno_freq'] = base_freq_df['term_freq']/n_models
    base_freq_df['ic'] = [-math.log(i,2) for i in base_freq_df['anno_freq']]
    return base_freq_df

# mica for pair of hp terms
def mica(hp1, hp2, anc_df):
    anc1 = ont.ancestors(hp1)
    anc1.insert(0, hp1)

    anc2 = ont.ancestors(hp2)
    anc2.insert(0, hp2)
    
    # overlapping anc terms between two hp terms
    anc_overlap = list(set(anc1).intersection(set(anc2)))
    
    mica = anc_df[anc_df['hp'].isin(anc_overlap)]['ic'].max()
    
    return mica

# df with terms of one model as rows and terms of the other as columns
# compute mica for each pair of terms
def mica_df(model_id1, model_id2, base_df, anc_df):
    model1 = base_df[base_df['model_id']== model_id1].reset_index()
    model2 = base_df[base_df['model_id']== model_id2].reset_index()
    mica_df = pd.DataFrame(index=model1['hp'],columns=model2['hp'])

    for i in range(len(mica_df.index)):
        for j in range(len(mica_df.columns)):
            hp1 = mica_df.index[i]
            hp2 = mica_df.columns[j]
            mica_df.loc[hp1,hp2] = mica(hp1,hp2,anc_df)
        
    return mica_df

# sim max is max of all rows + max of all cols /2 
# see PMID 31104773
def sim_max(mica_df):
    row_max = (mica_df.max(axis=1).values).sum()
    col_max = (mica_df.max(axis=0).values).sum()
    sim_max = (row_max+col_max)/2
    return sim_max

def sim_av_from_precomp(model_id1, model_id2, base_df, precomp_mica):

    model1 = base_df[base_df['model_id']== model_id1].reset_index()
    model2 = base_df[base_df['model_id']== model_id2].reset_index()

    mica_cols = precomp_mica[precomp_mica['hp2'].isin(model1['hp'])]
    mica_cols = mica_cols[mica_cols['hp1'].isin(model2['hp'])]
    m = len(set(mica_cols['hp1']))
    
    mica_rows = precomp_mica[precomp_mica['hp1'].isin(model1['hp'])]
    mica_rows = mica_rows[mica_rows['hp2'].isin(model2['hp'])]
    n = len(set(mica_rows['hp1']))
    
    
    mica_col_lst = []
    for i in set(mica_cols['hp1']):
        mica = mica_cols[mica_cols['hp1']==i]['mica'].max()
        mica_col_lst.append({'hp1': i, 'mica': mica})
    mica_col = pd.DataFrame(mica_col_lst).merge(model2, how='left',left_on='hp1',right_on='hp')
    mica_col['weighted_mica'] = mica_col['mica']*mica_col['model_freq']
    mica_col_mean = (mica_col['weighted_mica'].sum())/len(mica_col)
    
    mica_row_lst = []
    for i in set(mica_rows['hp1']):
        mica = mica_rows[mica_rows['hp1']==i]['mica'].max()
        mica_row_lst.append({'hp1': i, 'mica': mica})
    mica_row = pd.DataFrame(mica_row_lst).merge(model1, how='left',left_on='hp1',right_on='hp')
    mica_row['weighted_mica'] = mica_row['mica']*mica_col['model_freq']
    mica_row_mean = (mica_row['weighted_mica'].sum())/len(mica_col)


    return 0.5*(mica_col_mean+mica_row_mean)

def sim_df(base_df, precomp_mica, suffix1, suffix2):

    model_ids1 = [i for i in list(dict.fromkeys(base_df['model_id'])) if suffix1 in i]
    model_ids2 = [i for i in list(dict.fromkeys(base_df['model_id'])) if suffix2 in i]

    sim_df = pd.DataFrame(index=model_ids1,columns=model_ids2)
    
    for i in range(len(sim_df.index)):
        for j in range(len(sim_df.columns)):
            model_id1 = sim_df.index[i]
            model_id2 = sim_df.columns[j]
            sim_df.loc[model_id1,model_id2] = sim_av_from_precomp(model_id1, model_id2, base_df, precomp_mica)
            print (f'added sim_av {model_id1,model_id2}')
    sim_df = sim_df.apply(pd.to_numeric)    
    return sim_df

In [ ]:
def precompute_mica(anc_base):
    pre_lst = []
    for i in range(len(anc_base)):
        for j in range(len(anc_base)):
            k = (anc_base['hp'][i], anc_base['hp'][j])
            pre_lst.append(k)

    precompute_mica = pd.DataFrame(pre_lst, columns =['hp1','hp2'])

    precompute_mica['mica']  = precompute_mica.apply(lambda x: mica(x.hp1, x.hp2, anc_base), axis=1)
    precompute_mica['mica'] = precompute_mica['mica'].fillna(-0.0)
    
    return precompute_mica

In [ ]:
def load_pheno_csv(pheno_csv):
    phen_df = pd.read_csv(pheno_csv)
    phen_df['hp_base'] = [i.split(",") for i in phen_df['hp_base']]
    return phen_df

In [ ]:
cont_ddd = load_pheno_csv('path_to_content_text_base.csv')
mim_ddd = load_pheno_csv('path_to_mim_base.csv')

In [ ]:
base_df_cont_ddd = hp_base_df(cont_ddd)
# ic for all terms across set including propagated ancestors
anc_base_cont_ddd = ic_df(anc_freq_df(base_df_cont_ddd))

base_df_mim_ddd = hp_base_df(mim_ddd)
# ic for all terms across set including propagated ancestors
anc_base_mim_ddd = ic_df(anc_freq_df(base_df_mim_ddd))

In [ ]:
precompute_mica_cont_ddd = precompute_mica(anc_base_cont_ddd)
pickle.dump(precompute_mica_cont_ddd,open('path_to_precompute_cont_ddd.p','wb'))

precompute_mica_mim_ddd = precompute_mica(anc_base_mim_ddd)
pickle.dump(precompute_mica_mim_ddd,open('path_to_precompute_mim_ddd.p','wb'))

In [ ]:
sim_df_cont_ddd = sim_df(base_df_cont_ddd, precompute_mica_cont_ddd, '_content', '_ddd')
pickle.dump(sim_df_cont_ddd, open('path_to_cont_ddd_simdf.p','wb'))

sim_df_mim_ddd = sim_df(base_df_mim_ddd, precompute_mica_mim_ddd, '_mim', '_ddd')
pickle.dump(sim_df_mim_ddd, open('path_to_mim_ddd_simdf.p','wb'))